In [17]:
#selenium e webdriver automatizam ações no navegador
#pandas permite trabalhar com planilhas no Python

from selenium import webdriver
from selenium.webdriver.common.keys import Keys #permite digitação no navegador
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [18]:
lista_alunos = pd.read_excel("alunos.xlsx") #cria variável que recebe a planilha do excel
lista_alunos["RA"] = lista_alunos["RA"].astype(str) #converte a coluna RA em string
print(lista_alunos.info()) #mostra informações gerais da planilha

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   TURMA        26 non-null     object 
 1   ALUNO        26 non-null     object 
 2   LOGIN        26 non-null     object 
 3   RA           26 non-null     object 
 4   VERIFICAÇÃO  0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 1.1+ KB
None


In [19]:
navegador = webdriver.Chrome() #inicia o navegador
navegador.get("https://omnium.agilixbuzz.com/") #acessa a página

try:
    element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.ID, "mat-input-0"))) #aguarda até 10s, para localizar o elemento login
except:
    print("Falha ao localizar elemento")
    pass    

for linha in range(0, 26):
    try:
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[1]/div/div[1]/div/input').clear() #apaga os dados, evita concatenação
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[2]/div/div[1]/div/input').clear() #apaga os dados, evita concatenação
        
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[1]/div/div[1]/div/input').send_keys(lista_alunos["LOGIN"][linha]) #seleciona login e insere as credenciais
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[2]/div/div[1]/div/input').send_keys(lista_alunos["RA"][linha]) #seleciona senha e insere as credenciais
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[1]/div/div[1]/div/input').send_keys(Keys.ENTER) #aperta a tecla Enter

        element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "mat-focus-indicator mat-menu-trigger user-menu-button mat-button mat-button-base"))) #aguarda até 10s, para localizar o elemento menu

        navegador.find_element_by_xpath('/html/body/app-root/app-after-login/div/mat-sidenav-container/mat-sidenav-content/mat-toolbar/div[2]/button[2]').click() #clica no menu
        lista_alunos.loc[linha, "VERIFICAÇÃO"] = "Verificado" #registra na planilha que foi verificado

    except:
        lista_alunos.loc[linha, "VERIFICAÇÃO"] = "Falha na verificação"#registra na planilha que houve falha
        pass

    try:
        element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "mat-focus-indicator mat-menu-item ng-tns-c156-5e"))) #aguarda até 10s até para localizar o elemento sair
        navegador.find_element_by_class_name('mat-focus-indicator mat-menu-item ng-tns-c156-5').click #sair
    except:
        pass

display(lista_alunos)

In [ ]:
navegador.quit() #encerra o navegador
lista_alunos.to_excel("alunos verificados.xlsx", index = False) #salva as informações em uma nova planilha